In [1]:
import sys, cv2, dlib, time,math
import numpy as np


In [8]:
# convert Dlib shape detector object to list of tuples
def dlibLandmarksToPoints(shape):
  points = []
  for p in shape.parts():
    pt = (p.x, p.y)
    points.append(pt)
  return points

# detect facial landmarks in image
def getLandmarks(faceDetector, landmarkDetector, im, FACE_DOWNSAMPLE_RATIO = 1):
  points = []
  imSmall = cv2.resize(im,None,
                       fx=1.0/FACE_DOWNSAMPLE_RATIO, 
                       fy=1.0/FACE_DOWNSAMPLE_RATIO, 
                       interpolation = cv2.INTER_LINEAR)
  
  faceRects = faceDetector(imSmall, 0)
  
  if len(faceRects) > 0:
    maxArea = 0
    maxRect = None
    # TODO: test on images with multiple faces
    for face in faceRects:
      if face.area() > maxArea:
        maxArea = face.area()
        maxRect = [face.left(),
                   face.top(),
                   face.right(),
                   face.bottom()
                  ]
    
    rect = dlib.rectangle(*maxRect)
    scaledRect = dlib.rectangle(int(rect.left()*FACE_DOWNSAMPLE_RATIO),
                             int(rect.top()*FACE_DOWNSAMPLE_RATIO),
                             int(rect.right()*FACE_DOWNSAMPLE_RATIO),
                             int(rect.bottom()*FACE_DOWNSAMPLE_RATIO))
    
    landmarks = landmarkDetector(im, scaledRect)
    points = dlibLandmarksToPoints(landmarks)
  return points



In [9]:
# Returns the vector of indices of 3 points for each triangle
def calculateDelaunayTriangles(rect, points):

  # Create an instance of Subdiv2D
  subdiv = cv2.Subdiv2D(rect)
  for p in points:
    subdiv.insert((p[0], p[1]))

  # Get Delaunay triangulation
  triangleList = subdiv.getTriangleList()
  delaunayTri = []

  for t in triangleList:
    pt = []
    pt.append((t[0], t[1]))
    pt.append((t[2], t[3]))
    pt.append((t[4], t[5]))

    pt1 = (t[0], t[1])
    pt2 = (t[2], t[3])
    pt3 = (t[4], t[5])

    if rectContains(rect, pt1) and rectContains(rect, pt2) and rectContains(rect, pt3):
      ind = []
      # Find the index of each vertex in the points list
      for j in range(0, 3):
        for k in range(0, len(points)):
          if(abs(pt[j][0] - points[k][0]) < 1.0 and abs(pt[j][1] - points[k][1]) < 1.0):
            ind.append(k)

      if len(ind) == 3:
        delaunayTri.append((ind[0], ind[1], ind[2]))

  return delaunayTri

# Check if a point is inside a rectangle
def rectContains(rect, point):
  if point[0] < rect[0]:
    return False
  elif point[1] < rect[1]:
    return False
  elif point[0] > rect[2]:
    return False
  elif point[1] > rect[3]:
    return False
  return True

In [11]:
# Apply affine transform tp output image
def applyAffineTransform(src, srcTri, dstTri, size):
  warpMat = cv2.getAffineTransform(np.float32(srcTri), np.float32(dstTri))

  dst = cv2.warpAffine(src, warpMat, (size[0], size[1]), None,
             flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)

  return dst



# Warps and alpha blends triangular regions from img1 and img2 to img
def warpTriangle(img1, img2, t1, t2):
  # Find bounding rectangle for each triangle
  r1 = cv2.boundingRect(np.float32([t1]))
  r2 = cv2.boundingRect(np.float32([t2]))

  t1Rect = []
  t2Rect = []
  t2RectInt = []

  for i in range(0, 3):
    t1Rect.append(((t1[i][0] - r1[0]), (t1[i][1] - r1[1])))
    t2Rect.append(((t2[i][0] - r2[0]), (t2[i][1] - r2[1])))
    t2RectInt.append(((t2[i][0] - r2[0]), (t2[i][1] - r2[1])))
    
  mask = np.zeros((r2[3], r2[2], 3), dtype=np.float32)
  cv2.fillConvexPoly(mask, np.int32(t2RectInt), (1.0, 1.0, 1.0), 16, 0)
  img1Rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
  size = (r2[2], r2[3])
  img2Rect = applyAffineTransform(img1Rect, t1Rect, t2Rect, size)
  img2Rect = img2Rect * mask
  # Copy  to the output image
  img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] * ((1.0, 1.0, 1.0) - mask)
  img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] + img2Rect

In [13]:
if __name__ == '__main__' :

  modelPath = "C:/Users/Mayank Vikram/Documents/Computer Vision/week4/practice/shape_predictor_68_face_landmarks.dat"
    
  # initialize the dlib facial landmakr detector
  detector = dlib.get_frontal_face_detector()
  predictor = dlib.shape_predictor(modelPath)

  t = time.time()
  # Read images
  filename2 = 'C:/Users/Mayank Vikram/Documents/Computer Vision/week5/Practice/Face Swap Photos/Barney Stintion.jpg'
  filename1 = 'C:/Users/Mayank Vikram/Documents/Computer Vision/week5/Practice/Face Swap Photos/Pic1 (1).jpg'
  
  img1 = cv2.imread(filename1)
  img2 = cv2.imread(filename2)
  img1Warped = np.copy(img2)   
  
  # Read array of corresponding points
  points1 = getLandmarks(detector, predictor, img1)
  points2 = getLandmarks(detector, predictor, img2)    
  
  # Find convex hull
  hull1 = []
  hull2 = []

  hullIndex = cv2.convexHull(np.array(points2), returnPoints=False)
        
  for i in range(0, len(hullIndex)):
    hull1.append(points1[hullIndex[i][0]])
    hull2.append(points2[hullIndex[i][0]])
  
  
  # Find delanauy traingulation for convex hull points
  sizeImg2 = img2.shape    
  rect = (0, 0, sizeImg2[1], sizeImg2[0])
   
  dt = calculateDelaunayTriangles(rect, hull2)
  
  if len(dt) == 0:
    quit()
  
  # Apply affine transformation to Delaunay triangles
  for i in range(0, len(dt)):
    t1 = []
    t2 = []
    
    #get points for img1, img2 corresponding to the triangles
    for j in range(0, 3):
      t1.append(hull1[dt[i][j]])
      t2.append(hull2[dt[i][j]])
    
    warpTriangle(img1, img1Warped, t1, t2)

  print("Time taken for faceswap {:.3f} seconds".format(time.time() - t))
  tClone = time.time()

  # Calculate Mask for Seamless cloning
  hull8U = []
  for i in range(0, len(hull2)):
    hull8U.append((hull2[i][0], hull2[i][1]))
  
  mask = np.zeros(img2.shape, dtype=img2.dtype)  
  
  cv2.fillConvexPoly(mask, np.int32(hull8U), (255, 255, 255))
  # find center of the mask to be cloned with the destination image
  r = cv2.boundingRect(np.float32([hull2]))    
  
  center = ((r[0]+int(r[2]/2), r[1]+int(r[3]/2)))
      
  # Clone seamlessly.
  output = cv2.seamlessClone(np.uint8(img1Warped), img2, mask, center, cv2.NORMAL_CLONE)
  print("Time taken for seamless cloning {:.3f} seconds".format(time.time() - tClone))

  print("Total Time taken {:.3f} seconds ".format(time.time() - t))

  #cv2.imshow("Face Swapped before seamless cloning", np.uint8(img1Warped))
  #cv2.imshow("Face Swapped after seamless cloning", output)

  cv2.imwrite("faceswap_final.jpg", output)

  #cv2.waitKey(0)
  
  cv2.destroyAllWindows()


Time taken for faceswap 2.760 seconds
Time taken for seamless cloning 0.010 seconds
Total Time taken 2.770 seconds 
